In [1]:
import torch
import numpy as np
from swag import run_SWAG
from definitions import DATASETS
from swag_mod import run_MCSWAG
from noisy_adam import run_noisy_ADAM

In [2]:
seed = 556

In [3]:
DSELECTED = [
    'boston_housing',
    'concrete',
    'energy_heating_load',
    'power',
    'wine',
    'yacht',
    'kin8nm',
#     'naval_compressor_decay',
#     'protein',
#     'year_prediction_msd',
            ]
DSELECTED 

['boston_housing',
 'concrete',
 'energy_heating_load',
 'power',
 'wine',
 'yacht',
 'kin8nm']

In [4]:
for dataset_name in DSELECTED:
    print(f"Dataset: {dataset_name}")
    torch.manual_seed(seed)
    np.random.seed(seed)
    run_SWAG(dataset_name, weight_decay=0, train_model_flag=True)
    torch.manual_seed(seed)
    np.random.seed(seed)
    run_MCSWAG(dataset_name)
    torch.manual_seed(seed)
    np.random.seed(seed)
    run_noisy_ADAM(dataset_name)

Dataset: boston_housing
      SWAG Test | RMSE: 2.876, PICP: 0.353, MPIW:2.795
    MCSWAG Test | RMSE: 3.290, PICP: 0.569, MPIW:6.530 
dataset: boston_housing, rows: 506, columns: 13, range of x: [0.0, 711.0], range of y: [5.0, 50.0]
Finished Training. Best validation loss: 0.22774 in epoch 20
Noisy ADAM Test | RMSE: 2.708, PICP: 0.471, MPIW:3.357
Dataset: concrete
      SWAG Test | RMSE: 5.681, PICP: 0.835, MPIW:16.086


KeyboardInterrupt: 

In [ ]:
K = 10
S = 500
weight_decay = 0
multiplier = 2
tolerance = 0.05
# for dataset_name in ['boston_housing']:
# for dataset_name in ['yacht']:
# for dataset_name in ['year_prediction_msd']:
# for dataset_name in ['naval_compressor_decay']:
#     for dataset_name in DATASETS[:3]:
for dataset_name in DATASETS:
#     for dataset_name in ['naval_compressor_decay']:
#     if dataset_name in TOLERANCE:
#         continue
    run_SWAG(dataset_name, weight_decay=0, train_model_flag=False)
    for tolerance in np.linspace(0.05, 0.08, 8):
        print(f"""-----> TOLERANCE: {tolerance}""")
        x_train, y_train, x_test, y_test, _, _ = load_dataset(dataset_name, verbose=False)
        batch_size = x_train.shape[0]//9
        model = create_model(x_train, layer_dims=[50], verbose=False)
    #     train_model(x_train, y_train, x_test, y_test, model, dataset_name, lr=0.001, epochs=50000, verbose=False, 
    #                 batch_size=batch_size, weight_decay=weight_decay)
        model = load_model(model, f"best_model_weights-{dataset_name}.pth", verbose=False)
        y_pred = get_test_predictions(x_train, y_train, x_test, y_test, model)
    #     print(f"SGD RMSE: {RMSE(y_pred, y_test):.3f}")
#         print("---MOD---")
        theta_epoch = torch.nn.utils.parameters_to_vector(model.parameters()).detach().cpu().clone()
        lr_multipliers = torch.ones_like(theta_epoch).float()
        for rounds in range(10):
            try:
                model = load_model(model, f"best_model_weights-{dataset_name}.pth", verbose=False)
                theta_swa, sigma_diag, D, thetas = train_SWAG_mod(x_train, y_train, x_test, y_test, model, K, lr_multipliers,
                                                              verbose=False, lr=0.01, batch_size=batch_size, 
                                                              weight_decay=weight_decay, epochs=50)
                print(torch.diagonal(calculate_sigma(sigma_diag, D, K)).numpy().mean())
                weight_series = torch.stack(thetas).numpy()
#                 step_plot = weight_series.shape[1] // 5
                weight_series -= weight_series.mean(axis=0, keepdims=True)
                weight_series /= weight_series.std(axis=0, keepdims=True) + 1e-10
    #             plt.plot(weight_series[:,::step_plot], alpha=0.3)
    #             plt.show()
                coeffs = calculate_coeffs(weight_series, False)
                lr_multipliers[np.abs(coeffs) > tolerance] *= multiplier
                _, x_val, _, y_val = train_test_split(x_train, y_train, test_size=0.1)
                y_val_pred = model(torch.from_numpy(x_val).float().to(device)).detach().cpu().numpy()
                sigma_diag = torch.clamp(sigma_diag, min=1e-10)
                rmse_test, pcip_test, mpiw_test = sample_and_get_metrics(x_train, y_train, x_test, y_test, 
                                                                         model, theta_swa, sigma_diag, D, K, S)
                rmse_val, pcip_val, mpiw_val = sample_and_get_metrics(x_train, y_train, x_val, y_val, 
                                                                         model, theta_swa, sigma_diag, D, K, S)
            except Exception as e:
#                 raise e
                print("Some error")
